# Ag-Analytics - Crop Identification API - Demo

The Crop Identification API is a new machine learning product developed from our high accuracy, near real-time ground truth agriculture data, the historical weather and soil data, and high-resolution remote sensing imagery. This product provides a timely and accurate crop type identification for the most common cash crops in the U.S. at 8m spatial resolution. 

The Ag-Analytics provides the Crop Identification API service which a user can provide a polygon in specific formats and a year to retrieve the crop type of their land in the given year. Additionally, this service also provides a summary of remote sensing information of their fields and display as time series chart on our website interface.

### Required libraries

In [12]:
import requests
import json
import time
import os
from pandas.io.json import json_normalize
from collections import defaultdict
import pandas as pd
import zipfile, io
from IPython.display import Image

%matplotlib inline
%autosave 0

Autosave disabled


### Request Parameter Details:

####Request URL:  https://ag-analytics.azure-api.net/cropidentification

1). ***SHAPE*** (geometry, file/text, required): Desired area-of-interest. The structure of the geometry can be one of the following.
    
    i. JSON geometry objects returned by the arcgis rest api, (file/text) 
    
    ii. GEOJSON (file/text)
    
    iii. Shapefile (file)
       
2). ***CropSeason***(string): The year of interest to retrieve the crop type information of the area-of-interest.
    
    Parameter example: "2018"

3). ***ModelType(string)***: Classificaiton algorithums. This version of Crop Identification supports: Decision Tree and Neural Networks classification algorithum.

    Prameter example:'TREE' or 'NN'(Optional, default as 'NN)

### Request in Json

The Crop Identification API service takes the request in Json format. Below are examples formatted in Json. 

In [4]:
#Request Example 1

request ={
    "SHAPE": "{\"type\":\"Feature\",\"properties\":{},\"geometry\":{\"type\":\"Polygon\",\"coordinates\":[[[-100.953840994,38.5946753571],[-100.953832008,38.5948720599],[-100.953876941,38.5952162884],[-100.953957821,38.5955324152],[-100.953984781,38.5955745654],[-100.954029714,38.5957361407],[-100.954245394,38.5961716896],[-100.954452087,38.5964807873],[-100.95473966,38.5968179832],[-100.954910406,38.596965506],[-100.954910406,38.5969795557],[-100.954982299,38.5970287299],[-100.954982299,38.5970427797],[-100.955359739,38.5973378239],[-100.955377712,38.5973378239],[-100.955629338,38.5975134449],[-100.956042724,38.5977312143],[-100.956357257,38.5978646856],[-100.956707736,38.5979911318],[-100.957175042,38.598110553],[-100.957615388,38.5981878255],[-100.958055734,38.598229974],[-100.958514053,38.5982369988],[-100.958963386,38.5982088998],[-100.959412719,38.5981456769],[-100.959906984,38.5980332805],[-100.96023949,38.5979279087],[-100.960598957,38.5977874128],[-100.960976396,38.5976047676],[-100.961281942,38.597422122],[-100.961560528,38.5972113765],[-100.961578502,38.5972113765],[-100.961848101,38.5969865806],[-100.961982901,38.5968390579],[-100.962018848,38.5968179832],[-100.962189594,38.5966283107],[-100.962315407,38.5964526876],[-100.962405274,38.5964526876],[-100.962405274,38.5963894631],[-100.962369327,38.5963894631],[-100.962369327,38.5963754133],[-100.962513113,38.5961716896],[-100.96262994,38.5959539155],[-100.962800686,38.5955324152],[-100.962908526,38.595054712],[-100.962926499,38.5948580097],[-100.962890553,38.5948509846],[-100.962962446,38.5948650348],[-100.963025352,38.5947947839],[-100.962998393,38.5947877588],[-100.962989406,38.5947666835],[-100.962917513,38.5947526333],[-100.962935486,38.5947245329],[-100.962926499,38.5946894073],[-100.962881566,38.5947034575],[-100.962665886,38.5946823822],[-100.958541013,38.5946753571],[-100.9584152,38.5947175077],[-100.958316347,38.5947175077],[-100.958298374,38.5947034575],[-100.958154587,38.5946753571],[-100.953840994,38.5946753571]]]}}",
    "ScalarVariables": {"CropSeason": "2018"},
    "ModelType":"NN"
}

#Request Example 2

# request = {
#     "SHAPE": "{\"type\":\"Feature\",\"properties\":{},\"geometry\":{\"type\":\"Polygon\",\"coordinates\":[[[-93.7748032713,41.6148440891],[-93.7747312917,41.6148642376],[-93.7746593121,41.6149179668],[-93.7746683095,41.6149381153],[-93.7746413172,41.6149649799],[-93.7746233223,41.6150254252],[-93.7746233223,41.633700319],[-93.7748392611,41.633700319],[-93.7748392611,41.6511615865],[-93.7748842483,41.6513092587],[-93.7749742228,41.65136967],[-93.7751001871,41.6513830947],[-93.7751811642,41.6513629576],[-93.7752441463,41.6513226834],[-93.775298131,41.6512287103],[-93.7753251234,41.6513092587],[-93.7753611132,41.6513428205],[-93.7754420902,41.6513763823],[-93.7755410622,41.6513830947],[-93.7756580291,41.6513428205],[-93.7757300087,41.6512488474],[-93.7757300087,41.6445831177],[-93.7759549449,41.6445831177],[-93.7759549449,41.6150052768],[-93.7759279526,41.6149313992],[-93.7758919628,41.6148911022],[-93.7757749959,41.6148440891],[-93.775676024,41.6148440891],[-93.7756130418,41.6148642376],[-93.7755410622,41.6149179668],[-93.7755500596,41.6149313992],[-93.7755050724,41.6149918445],[-93.7754960749,41.6149448315],[-93.7754330928,41.6148776699],[-93.7753431183,41.6148440891],[-93.7752351489,41.6148440891],[-93.7751361769,41.6148843861],[-93.7751361769,41.6148978184],[-93.7751001871,41.6149179668],[-93.7751091846,41.6149313992],[-93.7750821922,41.6149851284],[-93.7750641973,41.6149918445],[-93.7750551999,41.6149448315],[-93.7749922177,41.6148776699],[-93.7749022432,41.6148440891],[-93.7748032713,41.6148440891]]]}}",
#     "ScalarVariables": {
#         "CropSeason": "2018"
#     },
#     "ModelType":"NN"
# }

### API Function

In [5]:
def CropIdentification(request):
    try:
        url = 'https://ag-analytics.azure-api.net/cropidentification'
        
        response = requests.post(url, json = request,headers = {}).json()
        
        print(response)
        
        return response
    
    except Exception as e:
        print(e)
        raise e
                             

### Calling API Function and Displaying Response


In [6]:
CropIDresponse = CropIdentification(request)

{'feature_averages': [{'Key': 'Greenweek18', 'Value': 1138.131570659558}, {'Key': 'Redweek18', 'Value': 1903.1859603449168}, {'Key': 'SWIR2week18', 'Value': 2391.667558326072}, {'Key': 'SWIR1week18', 'Value': 3338.836322033022}, {'Key': 'NIRweek18', 'Value': 2970.0740078383446}, {'Key': 'Blueweek18', 'Value': 779.824844896861}, {'Key': 'Greenweek20', 'Value': 2254.270159289066}, {'Key': 'Redweek20', 'Value': 2480.602444221482}, {'Key': 'SWIR2week20', 'Value': 17.602483063970034}, {'Key': 'SWIR1week20', 'Value': 3808.579113377245}, {'Key': 'NIRweek20', 'Value': 3536.6660938053815}, {'Key': 'Blueweek20', 'Value': 1975.3829592151317}, {'Key': 'Greenweek22', 'Value': 1013.7187401697631}, {'Key': 'Redweek22', 'Value': 1324.5397082727213}, {'Key': 'SWIR2week22', 'Value': 2392.3059762302105}, {'Key': 'SWIR1week22', 'Value': 3199.2655615880494}, {'Key': 'NIRweek22', 'Value': 2412.510195847856}, {'Key': 'Blueweek22', 'Value': 733.3521051359988}, {'Key': 'Greenweek24', 'Value': 928.849268847287}

In [ ]:
#Response Example 1
CropIDresponse = {'feature_averages': [{'Key': 'Greenweek18', 'Value': 1138.131570659558}, {'Key': 'Redweek18', 'Value': 1903.1859603449168}, {'Key': 'SWIR2week18', 'Value': 2391.667558326072}, {'Key': 'SWIR1week18', 'Value': 3338.836322033022}, {'Key': 'NIRweek18', 'Value': 2970.0740078383446}, {'Key': 'Blueweek18', 'Value': 779.824844896861}, {'Key': 'Greenweek20', 'Value': 1007.8605842345153}, {'Key': 'Redweek20', 'Value': 2480.602444221482}, {'Key': 'SWIR2week20', 'Value': 17.602483063970034}, {'Key': 'SWIR1week20', 'Value': 3808.579113377245}, {'Key': 'NIRweek20', 'Value': 3536.6660938053815}, {'Key': 'Blueweek20', 'Value': 1975.3450576969717}, {'Key': 'Greenweek22', 'Value': 993.7888856711144}, {'Key': 'Redweek22', 'Value': 1324.5397082727213}, {'Key': 'SWIR2week22', 'Value': 2392.3059762302105}, {'Key': 'SWIR1week22', 'Value': 3199.2655615880494}, {'Key': 'NIRweek22', 'Value': 2412.510195847856}, {'Key': 'Blueweek22', 'Value': 733.3521051359988}, {'Key': 'Greenweek24', 'Value': 928.849268847287}, {'Key': 'Redweek24', 'Value': 1069.7853810301349}, {'Key': 'SWIR2week24', 'Value': 1066.0881596873523}, {'Key': 'SWIR1week24', 'Value': 1902.9297659572896}, {'Key': 'NIRweek24', 'Value': 3292.298120071036}, {'Key': 'Blueweek24', 'Value': 643.9180616872748}, {'Key': 'Greenweek26', 'Value': 571.9230231878212}, {'Key': 'Redweek26', 'Value': 512.0971963633662}, {'Key': 'SWIR2week26', 'Value': 577.5711350554554}, {'Key': 'SWIR1week26', 'Value': 1254.004163243775}, {'Key': 'NIRweek26', 'Value': 3535.8988991057713}, {'Key': 'Blueweek26', 'Value': 344.2115098055459}, {'Key': 'Greenweek28', 'Value': 457.9029104885397}, {'Key': 'Redweek28', 'Value': 379.21622211189487}, {'Key': 'SWIR1week28', 'Value': 677.788417488776}, {'Key': 'SWIR2week28', 'Value': 14.881174763569534}, {'Key': 'NIRweek28', 'Value': 3721.192446778267}, {'Key': 'Blueweek28', 'Value': 236.699024285531}, {'Key': 'Greenweek30', 'Value': 384.7649838193378}, {'Key': 'Redweek30', 'Value': 283.1936926800272}, {'Key': 'SWIR2week30', 'Value': 305.1543742073506}, {'Key': 'SWIR1week30', 'Value': 962.4707162781339}, {'Key': 'NIRweek30', 'Value': 3591.1494805204393}, {'Key': 'Blueweek30', 'Value': 204.9539935971929}, {'Key': 'Greenweek32', 'Value': 375.24875445591255}, {'Key': 'Redweek32', 'Value': 279.2847222138893}, {'Key': 'SWIR2week32', 'Value': 299.5264928749243}, {'Key': 'SWIR1week32', 'Value': 994.7986702873936}, {'Key': 'NIRweek32', 'Value': 3262.5500419977257}, {'Key': 'Blueweek32', 'Value': 216.0730864324732}, {'Key': 'Greenweek34', 'Value': 427.83056221801525}, {'Key': 'Redweek34', 'Value': 368.11916439871254}, {'Key': 'SWIR2week34', 'Value': 805.5621165630723}, {'Key': 'SWIR1week34', 'Value': 1417.8394552018106}, {'Key': 'NIRweek34', 'Value': 3102.557304281368}, {'Key': 'Blueweek34', 'Value': 266.59034645931484}, {'Key': 'Greenweek36', 'Value': 749.1587725403178}, {'Key': 'Redweek36', 'Value': 727.4151962193739}, {'Key': 'SWIR2week36', 'Value': 194.33282355324758}, {'Key': 'SWIR1week36', 'Value': 628.7585803430337}, {'Key': 'NIRweek36', 'Value': 2956.2606778525223}, {'Key': 'Blueweek36', 'Value': 621.9698331028843}, {'Key': 'GDD3', 'Value': 40.4415}, {'Key': 'Precipitation3', 'Value': 11.524}, {'Key': 'GDD4', 'Value': 88.57439999999998}, {'Key': 'Precipitation4', 'Value': 25.41900000000001}, {'Key': 'GDD5', 'Value': 567.5903999999999}, {'Key': 'Precipitation5', 'Value': 70.78399999999999}, {'Key': 'GDD6', 'Value': 808.3584}, {'Key': 'Precipitation6', 'Value': 152.91100000000003}, {'Key': 'GDD7', 'Value': 216.13589999999996}, {'Key': 'Precipitation7', 'Value': 1.4809999999999999}, {'Key': 'GDD8', 'Value': 0.0}, {'Key': 'Precipitation8', 'Value': 0.0}], 'raster_filename': 'result_cropidraster_20191126_232751_3827.tif', 'rasterinfo': {'CellSize': [0.0001, -0.0001], 'CoordinateSystem': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]', 'Extent': '-100.963025352, 38.5946369988, -100.953925352, 38.5982369988', 'Legend': [{'Area': 4.969644751890395, 'AreaPercent': '3.82 %', 'AreaUnit': 'ac', 'Count': 101, 'CountAllPixels': 2645, 'CropID': 1, 'CropName': 'CORN_WET', 'color': '#7fb367'}, {'Area': 125.17600246345707, 'AreaPercent': '96.18 %', 'AreaUnit': 'ac', 'Count': 2544, 'CountAllPixels': 2645, 'CropID': 4, 'CropName': 'WHEAT_HRD_RD_SPR', 'color': '#b1de70'}], 'pngb64': 'data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAFsAAAAkCAYAAADmQkaNAAABDklEQVR4nO3YyxGCMBSF4ZCxBRuwIVuxFFuxEXZsbcAC2LrSwZiEm8c9AXL+rTOgH2fi6GA21ON5e2lc93q5nzWum9rQ8uZauJJaPAA4dkvgUCh4CPYWgUNpwqti7wnZTQNdBXvPyG410atiHwnZrQZ6FewjI7uVoNvSm/cEbUzZ581edm/IbjkLz8LuHXpZCnryMULo31I8krAJ7U/qIsYmdDyJjwib0LLWnFaxCZ1WzCuKTei8Qm5BbEKX5fMr/gXJ5Hmxueo6uY5/2ITWi8eIcsvx2tALrH5cNqDPiIkN7PsXK48Q3cZp5rKRERuYNYZHCCouGxixgREbGLGBWX454uKygZ3GaW79HrrpDQsadk4r1itjAAAAAElFTkSuQmCC'}}

### Output Summary

In [7]:
df=defaultdict(list)
for value in CropIDresponse['rasterinfo']['Legend']:
    df['CropName'].append(value['CropName'])
    df['Area'].append(int(value['Area']))
    df['AreaUnit'].append(value['AreaUnit'])
    df['AreaPercent'].append(value['AreaPercent'])
    df['Color'].append(value['color'])
cropdf=pd.DataFrame.from_dict(df)
cropdf = cropdf.sort_values(by = ['Area'], ascending=False).style.applymap(lambda x:"background-color: %s"%x, subset=['Color'])
cropdf

,CropName,Area,AreaUnit,AreaPercent,Color
1,WHEAT_HRD_RD_SPR,125,ac,96.18 %,#d93757
0,CORN_WET,4,ac,3.82 %,#a1d822


### Displaying Output Image

In [8]:
#Image
cropImage = CropIDresponse['rasterinfo']['pngb64']

Image(url = cropImage, width= 400, height =200)


# GET Request

GET Request for downloading the POST requested file to local machine.

In [14]:
# Specify local path where file will be downloaded.
local_path = r"C:\<Path to download directory>" # E.g., r"C:\Users\John_Doe\Documents\rasters"

# Create GET request payload.
values = {'filename': CropIDresponse['raster_filename']}

In [15]:
def cropID_get(values, local_path):
    try:
        url = 'https://ag-analytics.azure-api.net/cropidentification/'
    
        download_path = os.path.join(local_path, values['filename'])
     
        response = requests.get(url, params=values)
        open(download_path, 'wb').write(response.content)
        
        print(response.url)

        return response
    
    except Exception as e:
        print(e)
        raise e

In [16]:
cropID_get(values, local_path)

https://ag-analytics.azure-api.net/cropidentification/?filename=result_cropidraster_20191204_231112_4148.tif


<Response [200]>

To experience the usage of this service or other services provided by Ag-Analytics, please go to our API website page:
    
https://analytics.ag/ApiDeveloper